---
## 💾 Reading the channels and frames from Nikon microscope images with `.nd2` format

In [ ]:
## Requirements
#!pip install nd2 xrray scipy netCDF4==1.6.4

In [ ]:
## libraries
import logging
logging.getLogger().setLevel(logging.INFO)

import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname,splitext
from os import makedirs

import nd2 
import numpy as np

In [ ]:
## parameters
input_path=None
output_path=None

renames={'P': 'well', 'C': 'channel'} ## rename dim labels

## Input

In [ ]:
f1 = nd2.ND2File(input_path)

## Image info

In [ ]:
print(('\n'+'-'*50+'\n').join([f"{k}\n{'-'*50}\n{v}" for k,v in f1.text_info.items()]))

### Dimensions

In [ ]:
print(f1.sizes)

### Channel names

In [ ]:
channels={d.channel.index:d.channel.name.lower() for d in f1.metadata.channels}
logging.info(channels)

### Dimension names

In [ ]:
print(f1.metadata.contents)

In [ ]:
## alt method for getting the lables
{getattr(f1.metadata.contents,s):s.replace('Count','') for s in dir(f1.metadata.contents) if not s.startswith('_')}

In [ ]:
## alt method for getting the lables
{getattr(f1.attributes,k):k.replace('Count','') for k in dir(f1.attributes) if not k.startswith('_') and k.endswith('Count') and not k.startswith('component')}

In [ ]:
dims={renames[k]:v for k,v in f1.sizes.items() if not k in ['X','Y']}
dims

## Output

In [ ]:
## 
da=f1.to_xarray()
da=da.rename(renames)
da

## Save images

In [16]:
## convert potenntially incompatible datatypes in metadata to `str` 
da=da.assign_attrs(
    metadata=str(da.attrs['metadata']),
)

In [ ]:
da.to_netcdf(output_path)